# Dask jobqueue example

## What is Dask jobqueue? (<https://jobqueue.dask.org/>)

* deploys Dask workers on typical HPC job queueing systems

## Monte-Carlo estimate with multiple Dask batch job workers

We define a Dask jobqueue cluster with Dask workers that each have 8 CPUs and 48 GB of memory.

In [1]:
import dask, dask.distributed
import dask_jobqueue

In [2]:
cluster = dask_jobqueue.SLURMCluster(

    # Dask worker size
    cores=8, memory='48GB',
    processes=1, # Dask workers per job
    
    # SLURM job script things
    queue='cluster', walltime='00:15:00',
    
    # Dask worker network and temporary storage
    interface='ib0', local_directory='$TMPDIR'
)

client = dask.distributed.Client(cluster)
cluster.scale(jobs=1)

In [3]:
client

Client Scheduler: tcp://172.18.4.11:45813 Dashboard: http://172.18.4.11:8787/status,Cluster Workers: 1 Cores: 8 Memory: 48.00 GB


### What is a jobqueue cluster?
The above is all we need to specify to run the computation on compute node Dask workers. 
Let's have a look at what's happening in the background.

In [4]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             47402   cluster dask-wor smomw122  R       0:46      1 neshcl214 


In [5]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p cluster
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=45G
#SBATCH -t 00:15:00

/gxfs_home/geomar/smomw122/miniconda3/envs/dask_jobqueue_workshop/bin/python -m distributed.cli.dask_worker tcp://172.18.4.11:45813 --nthreads 8 --memory-limit 48.00GB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0 --protocol tcp://



In [6]:
client

Client Scheduler: tcp://172.18.4.11:45813 Dashboard: http://172.18.4.11:8787/status,Cluster Workers: 1 Cores: 8 Memory: 48.00 GB


### Let's scale up the cluster

In [7]:
cluster.scale(jobs=8)

In [8]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             47403   cluster dask-wor smomw122  R       0:01      1 neshcl214 
             47404   cluster dask-wor smomw122  R       0:01      1 neshcl214 
             47405   cluster dask-wor smomw122  R       0:01      1 neshcl203 
             47406   cluster dask-wor smomw122  R       0:01      1 neshcl203 
             47407   cluster dask-wor smomw122  R       0:01      1 neshcl203 
             47408   cluster dask-wor smomw122  R       0:01      1 neshcl216 
             47409   cluster dask-wor smomw122  R       0:01      1 neshcl216 
             47402   cluster dask-wor smomw122  R       0:54      1 neshcl214 


In [9]:
client

Client Scheduler: tcp://172.18.4.11:45813 Dashboard: http://172.18.4.11:8787/status,Cluster Workers: 8 Cores: 64 Memory: 384.00 GB


### From here everything is the same as with LocalCluster

In [10]:
import numpy, dask.array

def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_shape[0] / number_of_chunks), 2)
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    # display(xy)
    
    return pi

### Let's calculate again...

In [11]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1415805632
   pi error: 1.2090389793328882e-05

CPU times: user 210 ms, sys: 16.2 ms, total: 226 ms
Wall time: 897 ms


In [12]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=250) # 100 GB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14160028672
   pi error: 7.633130207018723e-06

CPU times: user 752 ms, sys: 84.5 ms, total: 836 ms
Wall time: 5.4 s


In [13]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141599929856
   pi error: 7.276266206890369e-06

CPU times: user 6.65 s, sys: 531 ms, total: 7.18 s
Wall time: 49.7 s


### We can easily scale down the cluster

In [14]:
cluster.scale(jobs=2)

### And we can scale up the cluster whenever needed

In [15]:
cluster.scale(jobs=32)

In [16]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             47410   cluster dask-wor smomw122  R       0:07      1 neshcl214 
             47411   cluster dask-wor smomw122  R       0:07      1 neshcl214 
             47412   cluster dask-wor smomw122  R       0:07      1 neshcl203 
             47413   cluster dask-wor smomw122  R       0:07      1 neshcl203 
             47414   cluster dask-wor smomw122  R       0:07      1 neshcl203 
             47415   cluster dask-wor smomw122  R       0:07      1 neshcl216 
             47416   cluster dask-wor smomw122  R       0:07      1 neshcl216 
             47417   cluster dask-wor smomw122  R       0:07      1 neshcl216 
             47418   cluster dask-wor smomw122  R       0:07      1 neshcl222 
             47419   cluster dask-wor smomw122  R       0:07      1 neshcl222 
             47420   cluster dask-wor smomw122  R       0:07      1 neshcl222 
             47421   cluster dask-wor smomw12

In [20]:
client

Client Scheduler: tcp://172.18.4.11:45813 Dashboard: http://172.18.4.11:8787/status,Cluster Workers: 31 Cores: 248 Memory: 1.49 TB


### Let's calculate again...

In [21]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141592736832
   pi error: 8.324220690525408e-08

CPU times: user 2.87 s, sys: 190 ms, total: 3.06 s
Wall time: 15 s


In [22]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB


from 10000.0 GB randomly chosen positions
   pi estimate: 3.141590895808
   pi error: 1.7577817930103379e-06

CPU times: user 37.8 s, sys: 2.18 s, total: 40 s
Wall time: 2min 14s


### Note, we could also adaptively scale the jobqueue cluster!

Dask jobqueue is able to scale total worker number based on problem size. You can also specify a target duration.

In [23]:
cluster.adapt(
    minimum=2, maximum=48,
)

In [24]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1416021824
   pi error: 9.528810207104499e-06

CPU times: user 207 ms, sys: 400 ms, total: 608 ms
Wall time: 2.14 s


In [25]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141611273792
   pi error: 1.862020220677607e-05

CPU times: user 5.88 s, sys: 704 ms, total: 6.59 s
Wall time: 27 s


In [27]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB


from 10000.0 GB randomly chosen positions
   pi estimate: 3.1415928975616
   pi error: 2.4397180675705954e-07

CPU times: user 39.8 s, sys: 2.88 s, total: 42.7 s
Wall time: 1min 43s
